In [263]:
from extract import extract_dataset
import pandas as pd
from clean import *
import numpy as np

In [264]:
ADDRESS = "https://openpowerlifting.gitlab.io/opl-csv/files/openpowerlifting-latest.zip"

In [265]:
df = pd.read_csv(extract_dataset(ADDRESS))

/var/folders/tn/my7fmgyj3gx8811h_mwsw1mh0000gn/T/ipykernel_2167/1268291688.py:1: DtypeWarning: Columns (33,35,38) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(extract_dataset(ADDRESS))


In [266]:
replace_lift_null_data(df)
checks_best_lift(df)
calculate_total(df)
check_qualification(df)
df = rearrange_column(df, "Place")
df = drop_null_weights(df)
df = redefine_weight_classes(df)

/var/folders/tn/my7fmgyj3gx8811h_mwsw1mh0000gn/T/ipykernel_2167/1279690121.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_female["IPFWeightClassKg"] = pd.cut(df_female["IPFWeightClassKg"], bins = F_bins, labels = F_labels)
/var/folders/tn/my7fmgyj3gx8811h_mwsw1mh0000gn/T/ipykernel_2167/1279690121.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_male['IPFWeightClassKg'] = pd.cut(df_male['IPFWeightClassKg'], bins = M_bins, labels = M_labels)


In [285]:
fill_unknown_values(df, "Division", "Unknown")
fill_unknown_values(df, "MeetName", "UnknownMeetName")
fill_unknown_values(df, "MeetTown", "UnknownMeetTown")
fill_unknown_values(df, "MeetState", "UnknownMeetState")
fill_unknown_values(df, "MeetCountry", "UnknownMeetCountry")
df = df.loc[df["IPFWeightClassKg"].notna()]
fill_unknown_values(df, "Tested", "No")
assume_federation(df)
fill_unknown_values(df, "State", "N/A")
fill_unknown_values(df, "Country", "NotStated")
fill_lift_scores(df)

In [268]:
def redefine_age_classes(df):
    ipf_age_classes = {
        "sub-junior": "18-",
        "junior": "19-23",
        "open": "24-39",
        "masters 1": "40-49",
        "masters 2": "50-59",
        "masters 3": "60-69",
        "masters 4": "70+,"
    }
    pass

In [315]:
df.iloc[:,:15].info(verbose=True, show_counts=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2809555 entries, 0 to 2865620
Data columns (total 15 columns):
 #   Column          Non-Null Count    Dtype  
---  ------          --------------    -----  
 0   Name            2809555 non-null  object 
 1   Sex             2809555 non-null  object 
 2   Event           2809555 non-null  object 
 3   Equipment       2809555 non-null  object 
 4   Age             1789184 non-null  float64
 5   AgeClass        2038074 non-null  object 
 6   BirthYearClass  1855780 non-null  object 
 7   Division        2809555 non-null  object 
 8   BodyweightKg    2809555 non-null  float64
 9   WeightClassKg   2809555 non-null  object 
 10  Squat1Kg        2809555 non-null  float64
 11  Squat2Kg        2809555 non-null  float64
 12  Squat3Kg        2809555 non-null  float64
 13  Squat4Kg        2809555 non-null  float64
 14  Best3SquatKg    2809555 non-null  float64
dtypes: float64(7), object(8)
memory usage: 343.0+ MB


In [316]:
df[["Name", "Sex", "Event", "Age", "AgeClass", "BirthYearClass"]]

,Name,Sex,Event,Age,AgeClass,BirthYearClass
0,Alona Vladi,F,SBD,33.0,24-34,24-39
1,Galina Solovyanova,F,SBD,43.0,40-44,40-49
7,Tatyana Altunina,F,SBD,26.0,24-34,24-39
9,Ekaterina Glukhova,F,SBD,25.5,24-34,24-39
19,Galina Nesterova,F,D,19.0,18-19,19-23
...,...,...,...,...,...,...
2865616,Jari Martikainen,M,SBD,40.5,40-44,40-49
2865617,Ryan Stinn,M,SBD,34.5,24-34,24-39
2865618,Eumenes Souza,M,SBD,43.5,40-44,40-49
2865619,Jewgenij Kondraschow,M,SBD,32.5,24-34,24-39


In [322]:
df.loc[(df["AgeClass"].notna() & (df["BirthYearClass"]).notna())][["AgeClass", "BirthYearClass"]]

,AgeClass,BirthYearClass
0,24-34,24-39
1,40-44,40-49
7,24-34,24-39
9,24-34,24-39
19,18-19,19-23
...,...,...
2865616,40-44,40-49
2865617,24-34,24-39
2865618,40-44,40-49
2865619,24-34,24-39


In [321]:
df["BirthYearClass"].unique()

array(['24-39', '40-49', '19-23', '14-18', nan, '70-999', '50-59',
       '60-69'], dtype=object)